In [1]:
from pykafka import KafkaClient
import json 
import time
import uuid 
import pandas as pd
import numpy 
import time
from datetime import datetime, timedelta

input_file= pd.read_csv('~/projet_stream/simulator/mobile_data/kafka_minimal_df.csv')

#client = KafkaClient(hosts="localhost:9092")
#topic=client.topics['antennos']
#producer = topic.get_sync_producer()

#input_file = input_file[['t','PhoneId','x','y']]

input_file.PhoneId = input_file.PhoneId.astype(str)
#input_file.x = input_file.x/1000
#input_file.y = input_file.y/1000


base = datetime.now()
time_list = [base + timedelta(minutes=i) for i in range(len(input_file.t.unique()))]
#time_list = [time.mktime(t.timetuple()) for t in time_list]
time_list = [t.strftime("%m-%d-%Y %H:%M:%S") for t in time_list]

mergetime = pd.concat([pd.DataFrame(input_file.t.unique()),pd.DataFrame(time_list)],axis=1)
mergetime.columns =['t','timestamp']
input_file = input_file.merge(mergetime,on='t')
#input_file.timestamp = input_file.timestamp.astype(int)
input_file

,t,AntennaId,EventCode,PhoneId,x,y,TileId,timestamp
0,0,227,0,296,600995.281445,652785.571876,1147,05-01-2020 23:54:11
1,0,227,0,297,600995.281445,652785.571876,1147,05-01-2020 23:54:11
2,0,194,0,289,600384.426337,650135.919067,439,05-01-2020 23:54:11
3,0,180,0,257,600901.554282,651774.870814,876,05-01-2020 23:54:11
4,0,161,0,259,600203.078702,651730.080000,869,05-01-2020 23:54:11
...,...,...,...,...,...,...,...,...
1429,49,45,0,273,600005.273553,648885.714946,84,05-02-2020 00:43:11
1430,49,24,2,230,600448.807269,652805.610380,1168,05-02-2020 00:43:11
1431,49,23,2,297,600030.651665,651928.353400,921,05-02-2020 00:43:11
1432,49,23,2,296,600030.651665,651928.353400,921,05-02-2020 00:43:11


In [2]:
input_file.to_csv('../kafka_ingestion.csv',index=False,header=True)

In [3]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5 pyspark-shell'
import pyspark
import os
import re
from kafka import SimpleProducer, KafkaClient
from kafka import KafkaProducer
from pyspark.streaming import StreamingContext
from pyspark.sql import Column, DataFrame, Row, SparkSession
from pyspark.streaming.kafka import KafkaUtils
from pyspark import SparkConf, SparkContext
import time 
from pyspark.sql import SparkSession
from pyspark.sql.functions import unix_timestamp, col
from pyspark.sql import Window
from pyspark.sql.functions import *
spark = SparkSession.builder \
  .appName("Spark Structured Streaming from Kafka") \
  .getOrCreate()
sdfRides = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "antennes") \
  .option("startingOffsets", "latest") \
  .load() \
  .selectExpr("CAST(value AS STRING)")

from pyspark.sql.types import *
schema = StructType([StructField("t", IntegerType()),
                     StructField("AntennaId", IntegerType()),
                     StructField("EventCode", IntegerType()),
                     StructField("PhoneId", IntegerType()),
                     StructField("x", FloatType()),
                     StructField("y", FloatType()),
                     StructField("TileId", IntegerType()),
                     StructField("timestamp", StringType())])

In [4]:
#df = spark.read.csv('../kafka_minimal_df.csv',header=True,schema=schema)
df = spark.read.csv('../kafka_ingestion.csv',header=True,schema=schema)

In [5]:
from pyspark.sql.functions import unix_timestamp, col
df=df.withColumn('timestamp',unix_timestamp(df.timestamp, 'MM-dd-yyyy HH:mm:ss').cast(TimestampType()).alias("timestamp"))
sdfAntennes = df.where("EventCode!=1")

In [6]:
sdfAntennes = sdfAntennes.withColumn('x', sdfAntennes.x/1000).withColumn('y', sdfAntennes.y/1000)

In [7]:
sdfLoc = sdfAntennes.withWatermark("timestamp", "10 minutes").groupBy("PhoneId", window("timestamp", "1 minutes","1 minutes")).avg('x','y')

In [8]:
sdfLoc.show()

+-------+--------------------+-----------+-----------+
|PhoneId|              window|     avg(x)|     avg(y)|
+-------+--------------------+-----------+-----------+
|    253|[2020-05-01 23:57...|600.3348125|649.5083125|
|    246|[2020-05-02 00:10...|599.9878125|651.9330625|
|    273|[2020-05-02 00:19...| 600.098625|649.1323125|
|    296|[2020-05-02 00:20...|601.0140625|652.9041875|
|    270|[2020-05-02 00:32...|  600.03675| 651.380125|
|    246|[2020-05-02 00:32...|599.9878125|651.9330625|
|    296|[2020-05-02 00:40...| 600.030625| 651.928375|
|    270|[2020-05-02 00:00...| 600.831625|  650.57275|
|    284|[2020-05-02 00:04...|600.6393125|651.2441875|
|    284|[2020-05-02 00:26...|601.4441875|651.4429375|
|    284|[2020-05-02 00:31...|601.4441875|651.4429375|
|    292|[2020-05-02 00:36...|601.7929375| 650.162375|
|    245|[2020-05-02 00:36...|599.9878125|651.9330625|
|    246|[2020-05-02 00:42...|  600.00575|  652.09325|
|    270|[2020-05-02 00:33...|  600.03675| 651.380125|
|    241|[

In [9]:
sdfLoc2 = sdfLoc.select(col("PhoneId").alias("PhoneId"), col("window").alias("time"),col("avg(x)").alias("x"),col("avg(y)").alias("y"))

In [10]:
sdfLoc2.show()

+-------+--------------------+-----------+-----------+
|PhoneId|                time|          x|          y|
+-------+--------------------+-----------+-----------+
|    253|[2020-05-01 23:57...|600.3348125|649.5083125|
|    246|[2020-05-02 00:10...|599.9878125|651.9330625|
|    273|[2020-05-02 00:19...| 600.098625|649.1323125|
|    296|[2020-05-02 00:20...|601.0140625|652.9041875|
|    270|[2020-05-02 00:32...|  600.03675| 651.380125|
|    246|[2020-05-02 00:32...|599.9878125|651.9330625|
|    296|[2020-05-02 00:40...| 600.030625| 651.928375|
|    270|[2020-05-02 00:00...| 600.831625|  650.57275|
|    284|[2020-05-02 00:04...|600.6393125|651.2441875|
|    284|[2020-05-02 00:26...|601.4441875|651.4429375|
|    284|[2020-05-02 00:31...|601.4441875|651.4429375|
|    292|[2020-05-02 00:36...|601.7929375| 650.162375|
|    245|[2020-05-02 00:36...|599.9878125|651.9330625|
|    246|[2020-05-02 00:42...|  600.00575|  652.09325|
|    270|[2020-05-02 00:33...|  600.03675| 651.380125|
|    241|[

In [11]:
sdfLoc \
         .selectExpr("CAST(PhoneId AS STRING) AS key", "to_json(struct(*)) AS value").show()

+---+--------------------+
|key|               value|
+---+--------------------+
|253|{"PhoneId":253,"w...|
|246|{"PhoneId":246,"w...|
|273|{"PhoneId":273,"w...|
|296|{"PhoneId":296,"w...|
|270|{"PhoneId":270,"w...|
|246|{"PhoneId":246,"w...|
|296|{"PhoneId":296,"w...|
|270|{"PhoneId":270,"w...|
|284|{"PhoneId":284,"w...|
|284|{"PhoneId":284,"w...|
|284|{"PhoneId":284,"w...|
|292|{"PhoneId":292,"w...|
|245|{"PhoneId":245,"w...|
|246|{"PhoneId":246,"w...|
|270|{"PhoneId":270,"w...|
|241|{"PhoneId":241,"w...|
|284|{"PhoneId":284,"w...|
|270|{"PhoneId":270,"w...|
|268|{"PhoneId":268,"w...|
|242|{"PhoneId":242,"w...|
+---+--------------------+
only showing top 20 rows



In [12]:
sdf = sdfLoc.withColumn("key", lit(0))

In [13]:
sdf = sdfAntennes.withColumn("key", lit(0))

In [14]:
sdf.groupBy('key').agg(collect_list("x").alias("x"),collect_list("y").alias("y"), collect_list("PhoneId").alias("PhoneId"))

DataFrame[key: int, x: array<double>, y: array<double>, PhoneId: array<int>]

In [17]:
sdf

DataFrame[t: int, AntennaId: int, EventCode: int, PhoneId: int, x: double, y: double, TileId: int, timestamp: timestamp, key: int]

In [16]:
sdf.select(to_json(struct([col(c).alias(c) for c in sdf.columns]))).show()

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|structstojson(named_struct(NamePlaceholder(), t AS `t`, NamePlaceholder(), AntennaId AS `AntennaId`, NamePlaceholder(), EventCode AS `EventCode`, NamePlaceholder(), PhoneId AS `PhoneId`, NamePlaceholder(), x AS `x`, NamePlaceholder(), y AS `y`, NamePlaceholder(), TileId AS `TileId`, NamePlaceholder(), timestamp AS `timestamp`, NamePlaceholder(), key AS `key`))|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
sdf.selectExpr("CAST(key AS STRING) AS key", "to_json(struct(*)) AS value").show()

1100

In [18]:
from pyspark.sql.functions import *

In [8]:
df = df.where("EventCode!=1")

In [11]:
df=df.withColumn('x', df.x/1000).withColumn('y', df.y/1000)

In [14]:
df = df.limit(30)

In [16]:
df.show()

+---+---------+---------+-------+-----------+-----------+------+-------------------+
|  t|AntennaId|EventCode|PhoneId|          x|          y|TileId|          timestamp|
+---+---------+---------+-------+-----------+-----------+------+-------------------+
|  0|      227|        0|    296|600.9953125|652.7855625|  1147|2020-05-01 21:02:43|
|  0|      227|        0|    297|600.9953125|652.7855625|  1147|2020-05-01 21:02:43|
|  0|      194|        0|    289|600.3844375|650.1359375|   439|2020-05-01 21:02:43|
|  0|      180|        0|    257|600.9015625| 651.774875|   876|2020-05-01 21:02:43|
|  0|      161|        0|    259|600.2030625|651.7300625|   869|2020-05-01 21:02:43|
|  0|      159|        0|    230|601.1290625|653.1939375|  1256|2020-05-01 21:02:43|
|  0|      156|        0|    283|602.1035625|649.7421875|   348|2020-05-01 21:02:43|
|  0|      156|        0|    284|602.1035625|649.7421875|   348|2020-05-01 21:02:43|
|  0|      142|        0|    268|600.5644375|652.7293125|  1142|2

In [21]:
grouped_df = df.groupBy("PhoneId", window(df.timestamp, "1 minute")).mean('x','y')

In [30]:
grouped_df.select('window').show(20, False)

+------------------------------------------+
|window                                    |
+------------------------------------------+
|[2020-05-01 21:02:00, 2020-05-01 21:03:00]|
|[2020-05-01 21:02:00, 2020-05-01 21:03:00]|
|[2020-05-01 21:02:00, 2020-05-01 21:03:00]|
|[2020-05-01 21:02:00, 2020-05-01 21:03:00]|
|[2020-05-01 21:02:00, 2020-05-01 21:03:00]|
|[2020-05-01 21:02:00, 2020-05-01 21:03:00]|
|[2020-05-01 21:02:00, 2020-05-01 21:03:00]|
|[2020-05-01 21:02:00, 2020-05-01 21:03:00]|
|[2020-05-01 21:02:00, 2020-05-01 21:03:00]|
|[2020-05-01 21:02:00, 2020-05-01 21:03:00]|
|[2020-05-01 21:02:00, 2020-05-01 21:03:00]|
|[2020-05-01 21:02:00, 2020-05-01 21:03:00]|
|[2020-05-01 21:02:00, 2020-05-01 21:03:00]|
|[2020-05-01 21:02:00, 2020-05-01 21:03:00]|
|[2020-05-01 21:02:00, 2020-05-01 21:03:00]|
|[2020-05-01 21:02:00, 2020-05-01 21:03:00]|
|[2020-05-01 21:02:00, 2020-05-01 21:03:00]|
|[2020-05-01 21:02:00, 2020-05-01 21:03:00]|
|[2020-05-01 21:02:00, 2020-05-01 21:03:00]|
|[2020-05-

In [62]:
df_group = df.groupBy("PhoneId", window(df.timestamp, "1 minute","1 minute")).mean('x','y')

In [63]:
df_group.show(2)

+-------+--------------------+-----------+-----------+
|PhoneId|              window|     avg(x)|     avg(y)|
+-------+--------------------+-----------+-----------+
|    296|[2020-05-01 21:02...|600.9953125|652.7855625|
|    297|[2020-05-01 21:02...|600.9953125|652.7855625|
+-------+--------------------+-----------+-----------+
only showing top 2 rows



In [65]:
df_group = df_group.select(col("PhoneId").alias("PhoneId"), col("window").alias("time"),col("avg(x)").alias("x"),col("avg(y)").alias("y"))

In [47]:
send = df_group.selectExpr("CAST(window AS STRING) AS key", "to_json(struct(*)) AS value")

In [115]:
from pyspark.sql.functions import unix_timestamp, col
from pyspark.sql.types import TimestampType
df.select(unix_timestamp(df.timestamp, 'MM-dd-yyyy HH:mm:ss').cast(TimestampType()).alias("timestamp"))

+-------------------+
|          timestamp|
+-------------------+
|2020-05-01 17:03:05|
|2020-05-01 17:03:05|
|2020-05-01 17:03:05|
|2020-05-01 17:03:05|
|2020-05-01 17:03:05|
|2020-05-01 17:03:05|
|2020-05-01 17:03:05|
|2020-05-01 17:03:05|
|2020-05-01 17:03:05|
|2020-05-01 17:03:05|
|2020-05-01 17:03:05|
|2020-05-01 17:03:05|
|2020-05-01 17:03:05|
|2020-05-01 17:03:05|
|2020-05-01 17:03:05|
|2020-05-01 17:03:05|
|2020-05-01 17:03:05|
|2020-05-01 17:03:05|
|2020-05-01 17:03:05|
|2020-05-01 17:03:05|
+-------------------+
only showing top 20 rows



In [96]:
def generate_checkpoint(input_file):
    for i in range(0,input_file.t.max()):
        input_file_topush = input_file[input_file.t==i]
        data = {'x' : [] , 'y' : [] , 'text' : [],'time':[]}
        
        j=0
        while j < len(input_file_topush):
            data['time'].append(input_file['time'][j])
            data['text'].append(input_file['PhoneId'][j])
            data['x'].append(input_file['x'][j]/1000)
            data['y'].append(input_file['y'][j]/1000)
            j+=1
            
        message = json.dumps(data)
        print(message)
        producer.produce(message.encode('ascii'))
        time.sleep(3)
        
generate_checkpoint(input_file)

{"x": [0.6009952814449999, 0.6009952814449999, 0.6003844263370001, 0.6009015542819999, 0.6002030787020001, 0.6011290550689999, 0.6021035929669999, 0.6021035929669999, 0.600564455101, 0.6014626948470001, 0.6014626948470001, 0.6015149115959999, 0.6002164621539999, 0.6005096143499999, 0.6005096143499999, 0.6000087650870001, 0.600431357885, 0.6011365530939999, 0.6001109597030001, 0.6001109597030001, 0.6007149240709999, 0.5999173147260001], "y": [0.6527855718760001, 0.6527855718760001, 0.6501359190670001, 0.651774870814, 0.6517300799999999, 0.6531939500760001, 0.649742183146, 0.649742183146, 0.652729315347, 0.648964107958, 0.648964107958, 0.650068348954, 0.649458409534, 0.6514076580279999, 0.6514076580279999, 0.6508332775929999, 0.648956498195, 0.64974296542, 0.6518290850649999, 0.6518290850649999, 0.64909722544, 0.6487649372450001], "text": ["296", "297", "289", "257", "259", "230", "283", "284", "268", "241", "242", "234", "273", "291", "292", "270", "299", "276", "245", "246", "253", "28

NameError: name 'producer' is not defined

In [94]:
iput_file

NameError: name 'imput_file' is not defined

In [93]:
for i in range(0,input_file.t.max()):
    input_file_topush = input_file[input_file.t==i]
    data = {'x' : [] , 'y' : [] , 'text' : [],'time':[]}

    j=0
    while j < len(input_file_topush):
        data['time'].append(input_file['time'][j])
        data['text'].append(input_file['PhoneId'][j])
        data['x'].append(input_file['x'][j]/1000)
        data['y'].append(input_file['y'][j]/1000)
        j+=1

KeyError: 'time'

In [ ]:
def generate_checkpoint(input_file):
    for i in range(0,input_file.t.max()):
        input_file_topush = input_file[input_file.t==i]
        message = input_file_topush.to_json(orient='records')
        print(message)
        producer.produce(message.encode('ascii'))
        time.sleep(3)
        
generate_checkpoint(input_file)

[{"t":0,"PhoneId":"296","x":600.995281445,"y":652.785571876},{"t":0,"PhoneId":"297","x":600.995281445,"y":652.785571876},{"t":0,"PhoneId":"289","x":600.384426337,"y":650.135919067},{"t":0,"PhoneId":"257","x":600.901554282,"y":651.774870814},{"t":0,"PhoneId":"259","x":600.203078702,"y":651.73008},{"t":0,"PhoneId":"230","x":601.129055069,"y":653.193950076},{"t":0,"PhoneId":"283","x":602.103592967,"y":649.742183146},{"t":0,"PhoneId":"284","x":602.103592967,"y":649.742183146},{"t":0,"PhoneId":"268","x":600.564455101,"y":652.729315347},{"t":0,"PhoneId":"241","x":601.462694847,"y":648.964107958},{"t":0,"PhoneId":"242","x":601.462694847,"y":648.964107958},{"t":0,"PhoneId":"234","x":601.514911596,"y":650.068348954},{"t":0,"PhoneId":"273","x":600.216462154,"y":649.458409534},{"t":0,"PhoneId":"291","x":600.50961435,"y":651.407658028},{"t":0,"PhoneId":"292","x":600.50961435,"y":651.407658028},{"t":0,"PhoneId":"270","x":600.008765087,"y":650.833277593},{"t":0,"PhoneId":"299","x":600.431357885,"y":

[{"t":5,"PhoneId":"242","x":601.15316618,"y":651.525617877},{"t":5,"PhoneId":"241","x":601.15316618,"y":651.525617877},{"t":5,"PhoneId":"230","x":600.660717662,"y":651.332288618},{"t":5,"PhoneId":"289","x":600.384426337,"y":650.135919067},{"t":5,"PhoneId":"257","x":600.242429502,"y":652.408177728},{"t":5,"PhoneId":"230","x":600.660717662,"y":651.332288618},{"t":5,"PhoneId":"284","x":601.326463487,"y":650.448298249},{"t":5,"PhoneId":"283","x":601.326463487,"y":650.448298249},{"t":5,"PhoneId":"253","x":600.091015093,"y":649.791594036},{"t":5,"PhoneId":"297","x":600.373529431,"y":653.390314852},{"t":5,"PhoneId":"296","x":600.373529431,"y":653.390314852},{"t":5,"PhoneId":"234","x":601.514911596,"y":650.068348954},{"t":5,"PhoneId":"257","x":600.242429502,"y":652.408177728},{"t":5,"PhoneId":"268","x":601.159144747,"y":650.051779337},{"t":5,"PhoneId":"242","x":601.15316618,"y":651.525617877},{"t":5,"PhoneId":"241","x":601.15316618,"y":651.525617877},{"t":5,"PhoneId":"273","x":600.19574499,"y"